In [1]:
import pandas as pd
import datetime as dt

In [2]:
RAW_DATA_NAME = "data/QQQ.csv"
ANALYSIS_RESULT_DATA_NAME = "data/QQQ-result.csv"
ANALYSIS_FIXED_RESULT_DATA_NAME = "data/QQQ-result-fixed-cost.csv"
ANALYSIS_FIXED_SELL_RESULT_DATA_NAME = "data/QQQ-result-fixed-cost-sell.csv"

In [3]:
# 读取数据
def read_data(file_name=RAW_DATA_NAME) -> pd.DataFrame:
    return pd.read_csv(file_name)

# 输出数据
def write_data(data: pd.DataFrame, file_name: str = ANALYSIS_RESULT_DATA_NAME) -> None:
    return data.to_csv(file_name, index=False, float_format='%.4f')

# 判断是否为星期一
def is_monday(day: str) -> bool:
    return dt.datetime.strptime(day, '%Y-%m-%d').strftime('%A') == 'Monday'

# 从年数，回报倍数得到年化收益. e.g. 0.1 -> 10% 年化收益
def annual_return(num_of_year: int, gain: float, inflation=0) -> float:
    return pow(gain - inflation, 1 / num_of_year) - 1.0

In [4]:
data = read_data()

In [5]:
data

,OPEN,HIGH,LOW,CLOSE,Volume,Dividends,Stock Splits,CODES,DATES
0,51.53,51.84,51.45,51.68,53656100,0.0,0,QQQ,2011-01-27
1,51.67,51.72,50.14,50.37,123359900,0.0,0,QQQ,2011-01-28
2,50.44,50.76,50.06,50.62,70571300,0.0,0,QQQ,2011-01-31
3,51.00,51.68,50.94,51.56,76952900,0.0,0,QQQ,2011-02-01
4,51.46,51.67,51.38,51.48,48661000,0.0,0,QQQ,2011-02-02
...,...,...,...,...,...,...,...,...,...
2511,320.14,324.74,316.51,323.77,30728100,0.0,0,QQQ,2021-01-20
2512,325.15,327.13,323.73,326.36,25316900,0.0,0,QQQ,2021-01-21
2513,325.29,326.46,324.71,325.42,21609400,0.0,0,QQQ,2021-01-22
2514,328.91,330.32,321.41,328.11,39080600,0.0,0,QQQ,2021-01-25


In [6]:
def calculate_scheduled_investment(data: pd.DataFrame) -> ():
    shares = 10
    positions = [0.0]
    cost = [0.0]
    assets = [0.0]
    for i in range(1, len(data)):
        open_price = data.iloc[i]['OPEN']
        date = data.iloc[i]['DATES']
        if is_monday(date):
            current_cost = shares * open_price
            positions.append(positions[-1] + shares)
            cost.append(cost[-1] + current_cost)
        else:
            positions.append(positions[-1])
            cost.append(cost[-1])
        current_asset = open_price * positions[i]
        assets.append(current_asset)
    print(len(positions))
    return positions, cost, assets

In [7]:
def export_result() -> float:
    # 生成 {first_name}_QQQ-result.csv, 目标是跟QQQ-result-expected.csv一致
    # 在这里调用 calculate_scheduled_investment, 并且赋值到asset和cost
    # 最后返回十年的年化率
    positions, cost, assets = calculate_scheduled_investment(read_data())
    data = read_data()
    precentage = [0.0]
    for i in range(len(cost)):
        while cost[i] > 0:
            rate = assets[i] / cost[i]
            precentage.append(rate)
    add = pd.DataFrame([positions, cost, assets, precentage],
                       index=['POSITIONS','COST','ASSETS','PRECENTAGE'])
    results = pd.merge(data, add, axis=1)
    write_data(data=results, file_name="Ivory-QQQ-result-expected")
    asset = assets[-1]
    cost = cost[-1]
    return annual_return(10, asset / cost)